In [187]:
import pandas as pd
import tweepy #https://github.com/tweepy/tweepy
import csv
from datetime import datetime
import os
import re
import requests
import twitter
import lxml.html
import time

#Twitter API credentials
consumerkey = 'EV9ulEXQ23ZxK8CzQqpSH5S26'
consumersecret = 'AIEMTk197UwFQRl1IEqGmBgz8fONJoJaI6RjW59H9EvHX261cY'
accesskey = '1119208863574708224-R0rPmcbD8ReYeak6u4Qeu7Idv0cPeI'
accesssecret = 'pgl972ioi4RrWcxkyUmB7TvuNsr0nJOLRpL8oX2rgqyrE'

auth = tweepy.OAuthHandler(consumerkey, consumersecret)
auth.set_access_token(accesskey, accesssecret)
api = tweepy.API(auth)

ironhack = pd.read_csv('ironhack_twitter_community.csv')

In [5]:
ironhack.head()

,Unnamed: 0,id,screen_name,location,protected,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,default_profile,default_profile_image
0,0,562076241,AlbaNebotLopeez,Barcelona.,False,False,133,167,1,374,382,Tue Apr 24 12:48:02 +0000 2012,False,False
1,1,1068503088737198083,RafaelF03113877,"London, England",False,False,94,90,1,170,224,Fri Nov 30 13:52:42 +0000 2018,False,False
2,2,705435773,berelingo,NaN,False,False,256,122,3,18456,9590,Tue Oct 08 19:50:24 +0000 2013,False,False
3,3,21900494,tombentler,NaN,False,False,15,114,0,24,7,Wed Feb 25 18:42:56 +0000 2009,True,False
4,4,875355002256199682,YeahCool30,NaN,False,False,9,105,0,258,17,Thu Jun 15 14:11:10 +0000 2017,True,False


In [6]:
ironhack.describe()

,Unnamed: 0,id,followers_count,friends_count,listed_count,favourites_count,statuses_count
count,12205.000000,1.220500e+04,1.220500e+04,12205.000000,12205.000000,12205.000000,1.220500e+04
mean,6102.000000,1.435787e+17,1.916582e+03,1646.180090,62.505039,2891.769111,4.941899e+03
std,3523.424352,3.247707e+17,2.842752e+04,8315.578048,256.063195,11774.755447,2.221246e+04
min,0.000000,1.353200e+04,0.000000e+00,1.000000,0.000000,0.000000,0.000000e+00
25%,3051.000000,1.407636e+08,7.300000e+01,245.000000,1.000000,52.000000,1.340000e+02
50%,6102.000000,5.245893e+08,2.440000e+02,625.000000,9.000000,382.000000,7.930000e+02
75%,9153.000000,3.063833e+09,7.060000e+02,1450.000000,40.000000,1842.000000,3.307000e+03
max,12204.000000,1.232369e+18,2.878503e+06,401962.000000,9764.000000,597215.000000,1.049781e+06


In [211]:
ironhack_q3= ironhack[(ironhack['protected']==False)&(ironhack['statuses_count'] <= 1842)&(ironhack['statuses_count'] >= 382)]
ironhack_q3.shape

(3062, 14)

In [12]:
t = twitter.Api(
    consumer_key = consumerkey,
    consumer_secret = consumersecret,
    access_token_key = accesskey, 
    access_token_secret = accesssecret,
    tweet_mode='extended' # this ensures that we get the full text of the users' original tweets
)

In [221]:
def first_200(name): #working
    sample = t.GetUserTimeline(screen_name=name, count=200)
    return sample

def get_tweets(name):
    all_tweets = []
    sample = first_200(name)
    all_tweets.extend(sample)
    last_id = sample[-1].id
    for i in range(30):
        time.sleep(1)
        new = t.GetUserTimeline(screen_name=name, max_id=last_id-1)
        if len(new) > 0:
            all_tweets.extend(new)
            last_id = new[-1].id
        else:
            break
    
    return all_tweets

In [190]:
tweets = first_200('bernat_vazquez')
text = ""

for tweet in tweets : 
    if (tweet.lang =='en'):
        text += tweet.full_text
        
print(text)
len(tweets)

@nhqe welcome to @DeliverooDesignYesterday at #BAU Centro Universitario de Diseño de Barcelona, we mentored #Design students to find their first job. It was great to see so many talented &amp; passionate designers. You'll rock it 👌  

#deliveroo #productdesign https://t.co/VvakQdK8Xg@Deliveroo , we're hiring a few people to join our @DeliverooDesign Team in London. We're 👀 for: Senior #Experience_Manager, Principal #Product_Designers, Principal #User_Researcher &amp; Senior #Content_Designer (perm roles). Check our🔝blog  https://t.co/eh6FeyIQcm and jobs! https://t.co/lb7kTPDopCRT @DeliverooDesign: Our take on a developing discipline. Have you ever wondered...

by @welltendedtree 
https://t.co/sn3mgtaUb1Hi 👋, we’ve opened up 👀 recruitment for #Principal level #ProductDesigners at #Deliveroo in London. If you have folks in your networks who you think might be interested, I’d love to hear from them 👌
They can learn about the job here: https://t.co/XJT3tybPBH
#ProductDesign https://t.co/Q6

200

In [61]:
tweets_expanded = get_tweets('bernat_vazquez')
text = ""

for tweet in tweets_expanded : 
    if (tweet.lang =='en'):
        text += tweet.full_text
        
print(text)
len(tweets_expanded)

Yesterday at #BAU Centro Universitario de Diseño de Barcelona, we mentored #Design students to find their first job. It was great to see so many talented &amp; passionate designers. You'll rock it 👌  

#deliveroo #productdesign https://t.co/VvakQdK8Xg@Deliveroo , we're hiring a few people to join our @DeliverooDesign Team in London. We're 👀 for: Senior #Experience_Manager, Principal #Product_Designers, Principal #User_Researcher &amp; Senior #Content_Designer (perm roles). Check our🔝blog  https://t.co/eh6FeyIQcm and jobs! https://t.co/lb7kTPDopCRT @DeliverooDesign: Our take on a developing discipline. Have you ever wondered...

by @welltendedtree 
https://t.co/sn3mgtaUb1Hi 👋, we’ve opened up 👀 recruitment for #Principal level #ProductDesigners at #Deliveroo in London. If you have folks in your networks who you think might be interested, I’d love to hear from them 👌
They can learn about the job here: https://t.co/XJT3tybPBH
#ProductDesign https://t.co/Q60vCW4qXnRT @stuartfrisby: We’ve j

382

In [28]:
from watson_developer_cloud import PersonalityInsightsV3
import json 

url = 'https://api.eu-gb.personality-insights.watson.cloud.ibm.com/instances/0a14be52-588b-406b-be32-d69c5047a806'
api_key = 'VA5FxxE5asUu3h2hUWZw17iXqxIoshi90YdDjSJgQgbj'

In [29]:
service = PersonalityInsightsV3( url = url , iam_apikey = api_key , version ='2020-01-30' )

/home/jose/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  """Entry point for launching an IPython kernel.


In [62]:
#profile = service.profile(text,content_type = 'text/plain', consumption_preferences = True).get_result()

In [63]:
profile['warnings']

[]

In [64]:
print(json.dumps(profile, indent = 2))

{
  "word_count": 3952,
  "processed_language": "en",
  "personality": [
    {
      "trait_id": "big5_openness",
      "name": "Openness",
      "category": "personality",
      "percentile": 0.8124448363398795,
      "significant": true,
      "children": [
        {
          "trait_id": "facet_adventurousness",
          "name": "Adventurousness",
          "category": "personality",
          "percentile": 0.9919362801386431,
          "significant": true
        },
        {
          "trait_id": "facet_artistic_interests",
          "name": "Artistic interests",
          "category": "personality",
          "percentile": 0.5979262916448119,
          "significant": true
        },
        {
          "trait_id": "facet_emotionality",
          "name": "Emotionality",
          "category": "personality",
          "percentile": 0.1254305878604049,
          "significant": true
        },
        {
          "trait_id": "facet_imagination",
          "name": "Imagination",
      

In [65]:
"""

This means I can safely extract tweets from people who have at least 382 tweets
(>50% of the population)

"""

'\n\nThis means I can safely extract tweets from people who have at least 382 tweets\n(>50% of the population)\n\n'

In [86]:
#tweets_expanded = get_tweets('bernat_vazquez')
#text = ""

def clean_hashtags(hashtags):
    """
    Turns data with any number of hashtags like this - [Hashtag(Text='STEMonStation')] - to a list like this -
    ['STEMonStation']
    """
    cleaned = []
    if len(hashtags) >= 1:
        for i in range(len(hashtags)):
            cleaned.append(hashtags[i].text)        
    return cleaned



In [99]:
def full_tweet_text_hashtag(name) : 
    tweets_expanded = get_tweets(name)
    
    text = ''
    hashtags = []
    
    for tweet in tweets_expanded :
        if (tweet.lang =='en'):
            text += tweet.full_text
            hashtags.append(clean_hashtags(tweet.hashtags))
    output = [text, hashtags]
            
    return output
        
print(text)
len(tweets_expanded)

382

In [109]:
output = full_tweet_text_hashtag('katmryc')

In [110]:
output[0]

'I just signed a petition to tell the world’s biggest companies to reject cheap, disposable plastics and invest in reuse and refill systems. Join me: https://t.co/OvT505l0kj #ReuseRevolutionOur blue planet is under threat. But now governments are debating a new Global Ocean Treaty that could lead to a third of the world’s oceans being protected from destruction. Join the campaign here: https://t.co/KAZAHhTBgWRT @PDChina: Add a stroke to validity the equation. What would you do? https://t.co/xnW4jtVef8Cannot wait! https://t.co/SNw5jkEMfo@LeylaSeka @HarryStebbings Loved it! Thank you for sharing your learnings @LeylaSekaIn 2019, #ichooseplanet bymaking New Year’s Resolutions that reduce the amount of single-use plastic in my daily life to keep it from reaching the ocean. What’s your New Year’s Resolution? Take the pledge and pass it on. https://t.co/1I49WpT3v9RT @NASA: The Arctic Ocean\'s older, thicker blanket of sea ice is nearly gone. What remains is younger, thinner, more variable ic

In [83]:
print("Data in the created_at attribute looks like this:", tweets_expanded[1].created_at)
print("Data in the hashtags attribute looks like this:", tweets_expanded[1].hashtags)
print("Data in the urls attribute looks like this:", tweets_expanded[1].urls)
print("Data in the source attribute looks like this:", tweets_expanded[1].source)

Data in the created_at attribute looks like this: Mon Mar 02 18:12:46 +0000 2020
Data in the hashtags attribute looks like this: [Hashtag(Text='Experience_Manager'), Hashtag(Text='Product_Designers'), Hashtag(Text='User_Researcher'), Hashtag(Text='Content_Designer')]
Data in the urls attribute looks like this: [URL(URL=https://t.co/eh6FeyIQcm, ExpandedURL=https://www.deliveroo.design/)]
Data in the source attribute looks like this: <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>


In [191]:
def population_tweets_hashtags(names_column) :
    for x in names_column:
        output_list = [[full_tweet_text_hashtag(x)[0],full_tweet_text_hashtag(x)[1]] for x in names_column]
        output_list_text = [output_list[x][0] for x in range(len(names_column))]
        output_list_hash = [output_list[x][1] for x in range(len(names_column))]
        return output_list, output_list_text, output_list_hash



In [192]:
#samples = ironhack[ironhack['statuses_count'] == 382]['screen_name']
samples = ironhack[ironhack['statuses_count'] == 382]


In [194]:
main_test, main_test_text,main_test_hash = population_tweets_hashtags(samples['screen_name'])

In [195]:
main_test

[['@rodalies R4, 6:58 a Sants.@elizasjanes @WildpipM @MisElizaJane @tasyateles @RhiannonMFish @iamAvgeropoulos @linzzmorgan @WilliamMiller @ChelseyReist @RichardSHarmon @MrRickyWhittle @harmon_jess Happy bday ma lof 🧡 https://t.co/kwEnmJxRpvHappy bday to me! #21 🎂🍰 #bonacompanyia #tindreamicsperaaixo #graciespertot #HASHTAG ❤️😍 http://t.co/abDlxdOHxq"Baby when you\'re gone, I realise I\'m in love" http://t.co/dhZVCA6lA2POST AMD🔞 Mai defrauden. @ Piso Majo http://t.co/CTB871Ji6nMaremagnum. http://t.co/vZeQ7pZrXIAMD. @ Marhes Club http://t.co/qodnEGq2CxRT @_cronos2: "And all you touch and all you see\nis all your life will ever be.\nRun, rabbit run,\ndig that hole, forget the sun.""I\'m thinking "ok, is that the game you wanna play?" I\'ll find myself another girl." https://t.co/SZYZyHs8WGAnd tonighttttt...... yeaaaaaah! :D @ponsolga @anaquintasparra @MarianMeseguer @Maii212@MarianMeseguer @ponsolga @Maii212 of courseee! @anaquintasparra@fornerjunior conversacions que no saps com han arr

In [157]:
ironhack_test = ironhack[(ironhack['statuses_count'] >= 382)]

In [219]:
main, main_text,main_hash = population_tweets_hashtags(samples['screen_name'])

In [173]:
samples

,Unnamed: 0,id,screen_name,location,protected,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,default_profile,default_profile_image
0,0,562076241,AlbaNebotLopeez,Barcelona.,False,False,133,167,1,374,382,Tue Apr 24 12:48:02 +0000 2012,False,False
5890,5890,3726206296,bernat_vazquez,"Barcelona, Spain",False,False,251,460,18,387,382,Mon Sep 21 07:55:07 +0000 2015,False,False
6963,6963,4530240509,katmryc,"Miami,Fl",False,False,225,277,9,628,382,Sat Dec 19 01:05:19 +0000 2015,False,False


In [182]:
samples['apply_function'] = ironhack_q3['screen_name'].apply(lambda x: full_tweet_text_hashtag(x), axis=1 , result_type='expand')

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [213]:
ironhack_q3.shape

(3062, 14)

In [223]:
ironhack_q3['apply_function'] = ironhack_q3['screen_name'].apply(full_tweet_text_hashtag)

KeyboardInterrupt: 

In [224]:
ironhack_testers = ironhack_q3.sample(n=600)

In [225]:
ironhack_testers.head()

,Unnamed: 0,id,screen_name,location,protected,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,default_profile,default_profile_image
542,542,17449993,HeikoRauch,"Berlin, Germany",False,False,1022,1119,121,491,538,Mon Nov 17 20:42:47 +0000 2008,False,False
8135,8135,140469965,tadwalkar,"Cupertino, CA",False,False,120,144,12,84,549,Wed May 05 15:31:45 +0000 2010,True,False
1416,1416,817119614538809344,oldmanlopez,NaN,False,False,36,695,1,602,612,Thu Jan 05 21:24:32 +0000 2017,True,False
10700,10700,78621404,argami_,"Barcelona, Spain",False,False,205,473,23,157,1092,Wed Sep 30 14:58:08 +0000 2009,False,False
1546,1546,608778831,itzelgarmed,NaN,False,False,359,1658,2,551,1136,Fri Jun 15 02:23:17 +0000 2012,False,False


In [227]:
ironhack_testers.drop(columns = 'Unnamed: 0', inplace = True)
ironhack_testers.to_csv('ironhack_personality_testers.csv')

In [ ]:
ironhack_testers['apply_function1'] = ironhack_testers['screen_name'].apply(first_200)